<a href="https://colab.research.google.com/github/magnujo/DataScienceAlgorithms/blob/main/Lab4_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees
Tree-based models work by partitioning the input space into rectangular regions, and then assigning simple (typically constant) model to each region.

Tree-based models: Pro/Con
* `+` simple
* `+` powerful
* `+` human understandable
* `-` computationally expensive
* `-` The optimal tree cannot be found efficiently. Therefore trees are built using a heuristic.

In this notebook we will see how to make a simple decision tree.
There are a couple of different kinds, but we will focus on one called CART (Classification and Regression Trees).
CART is a nice to learn with, as it is easy to understand and implement.

In this notebook we focus on **classification** on the Iris dataset, a classical data science dataset.
The goal is to predict the type of iris plant ('setosa' 'versicolor' 'virginica') from sepal and petal dimensions.
With only relatively minor changes the decision tree we make here could also be used for regression as well.

![](https://www.wpclipart.com/plants/diagrams/plant_parts/petal_sepal_label.png)

## CART
A CART is a **binary tree**.
* Inner nodes (non-leaf nodes) have a splitting rule, which can be described as `if-then` rules.
If some condition is met, go to the left child, otherwise go to the right child.
     * Each inner node queries one of the features, and checks whether it fulfils some condition, typically a threshold or equality.
     * If the condition is met, go to the left child, otherwise go to the right child.
* Continue like this until a leaf node is reached. Once a leaf node is reached assign $x$ to the corresponding label or value.


The splitting rule should, at each node split the data, $S$, into $L_{d,\theta}$ (left) and $R_{d,\theta}$ (right) such that the information gain is maximized:
$$
G_{d,\theta}(S) = Q(S) - \frac{|L_{d,\theta}|}{|S|}Q(L_{d,\theta}) - \frac{|R_{d,\theta}|}{|S|}Q(R_{d,\theta})
$$
Where $Q$ is some impurity measure.
Until $|S|$ is smaller than some $s_{threshold}$ or all elements belong to the same class.
Once the tree is grown it is common to prune to reduce its complexity - but we won't do that in this notebook.


## Loading the Data
For this exercise we will combine the features and targets in one array.
This makes keeping track of the data a bit easier.

In [ ]:
# ! git pull

In [ ]:
## Load the data
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()

print(iris.DESCR)  # <-- Uncomment for more details on the data set

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [ ]:
data = np.concatenate([iris.data, iris.target[:,None]], axis=-1) # concatenate features and targets side by side

# Split into train and test
step_len = 10
data_test = data[::step_len,:]
np.random.shuffle(data_test)
data_train = np.delete(data, [step_len*i for i in range(data_test.shape[0])], axis=0)#.reshape([-1, data.shape[1]])

print('Feature Names:', iris.feature_names)
print('Target Names: ', iris.target_names)
print('data\t\t', data.shape)
print('data_test\t', data_test.shape)
print('data_train\t', data_train.shape)
print()

print('Example data')
print(data_train[::15,:])

Feature Names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target Names:  ['setosa' 'versicolor' 'virginica']
data		 (150, 5)
data_test	 (15, 5)
data_train	 (135, 5)

Example data
[[4.9 3.  1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [4.9 3.1 1.5 0.2 0. ]
 [6.4 3.2 4.5 1.5 1. ]
 [5.8 2.7 4.1 1.  1. ]
 [5.4 3.  4.5 1.5 1. ]
 [5.8 2.7 5.1 1.9 2. ]
 [7.7 3.8 6.7 2.2 2. ]
 [6.1 2.6 5.6 1.4 2. ]]


In [ ]:
data_test

array([[5.4, 3.7, 1.5, 0.2, 0. ],
       [6.9, 3.2, 5.7, 2.3, 2. ],
       [4.8, 3.1, 1.6, 0.2, 0. ],
       [6.5, 3.2, 5.1, 2. , 2. ],
       [7. , 3.2, 4.7, 1.4, 1. ],
       [5.9, 3.2, 4.8, 1.8, 1. ],
       [7.4, 2.8, 6.1, 1.9, 2. ],
       [5.5, 2.4, 3.8, 1.1, 1. ],
       [6.7, 3.1, 5.6, 2.4, 2. ],
       [5.1, 3.5, 1.4, 0.2, 0. ],
       [6.3, 3.3, 6. , 2.5, 2. ],
       [5. , 3.5, 1.3, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.5, 2.6, 4.4, 1.2, 1. ],
       [5. , 2. , 3.5, 1. , 1. ]])

In [ ]:
## Useful helper functions.
# Reading and understanding these is a good idea!

def class_counts(data):
    """Counts the number of each class label in the input dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in data:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [ ]:
print('The class distribution is as follows:')
print('Total:\t', class_counts(data))
print('Train:\t', class_counts(data_train))
print('Test:\t', class_counts(data_test))

The class distribution is as follows:
Total:	 {0.0: 50, 2.0: 50, 1.0: 50}
Train:	 {0.0: 45, 1.0: 45, 2.0: 45}
Test:	 {0.0: 5, 2.0: 5, 1.0: 5}


# Task 1: Impurity Measure

The first thing we need to do is to define an impurity measure.
There are a couple of different to choose between
 * https://en.wikipedia.org/wiki/Decision_tree_learning#Metrics

For this exercise we will use the Gini impurity (see Wikipedia).
The Gini impurity is the probability of picking two different objects when drawing two random samples from the population.

$$
I_G(p) = 1 - \sum_{i=1} p_i^2
$$


In [ ]:
c = class_counts(data)
for i in range(len(c)):
  print(c[i])
c

50
50
50


{0.0: 50, 1.0: 50, 2.0: 50}

In [ ]:
import math
def compute_impurity(data):
    """ Calculate the impurity using the Gini Impurity Metric.
        
        Write the function generally, such that it works with different numbers of classes.
    """
    # determine number of classes in data (last column)
    label_counts = class_counts(data)
    n = len(data)
    impurity = 1

    for i in range(len(label_counts)):
      impurity = impurity - pow(label_counts[i]/n,2)

    ## YOUR CODE HERE: Complete the implementation of the Gini Impurity calculation
    
    
    return impurity

print('Compare these results to the next text cell:')
print('compute_impurity(data_class)\t\t', compute_impurity(data_train))
print('compute_impurity(data_class[:90])\t', compute_impurity(data_train[:90]))
print('compute_impurity(data_class[:45])\t', compute_impurity(data_train[:45]))

Compare these results to the next text cell:
compute_impurity(data_class)		 0.6666666666666665
compute_impurity(data_class[:90])	 0.5
compute_impurity(data_class[:45])	 0.0


**Expected Result (Solution):**

compute_impurity(data_class)		 0.6666666666666665

compute_impurity(data_class[:90])	 0.5

compute_impurity(data_class[:45])	 0.0

# Task 2: Branching
> Estimated time: 30 minutes.

The next thing we need to be able to do is use the impurity measure to split the data.
A split is based on a `if-then` rule, here called a `Question`.
A question is associated with a coordinate $d\in \{1, ..., D\}$ (what feature are we asking about), and a threshold, $\theta$.
* E.g: if $x_d < \theta$ go to left child, otherwise go to right child.

Note that each feature is considered independently - therefore decision trees aren't affected by normalization.

Your task is to:
1. Finish the `match` method in the `Question` class. 
 * Be sure that it can handle both categorican and numerical data (even though we only use numerical data here).
1. Complete the `partition` function in the next cell.
 * Use `compute_impurity` and `Question` to do so.

In [ ]:
class Question:
    """ A Question is used to partition a dataset.
        This class just records a 'column number' (aka coordinate) and a
        'column value' (aka threshold). The 'match' method is used to compare
        the feature value in an example to the feature value stored in the
        question. 
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value
        self.value_is_numerical = is_numeric(self.value)

    def __repr__(self):
        # Helper method to print in a readable format.
        condition = "=="
        if self.value_is_numerical:
            condition = ">="
        return "Is %s %s %s?" % (
            iris.feature_names[self.column], condition, str(self.value))
        
    def match(self, example):
        """ Takes a single example (a single row) and compares 
            it with the feature value in this question.
            
            Returns: Bool
        """
        if example[self.column] >= self.value:
          return True
        else:
          return False
        
        

        
# Example of a Question:
print(Question(1, 3.2))

Is sepal width (cm) >= 3.2?


In [ ]:
def partition(rows, question):
    """ Partitions a dataset:
        For each row in the dataset, check if it matches the question. 
        If so, add it to 'true rows', otherwise, add it to 'false rows'.
        
        Returns: true_rows, false_rows
    """
    
    true_rows, false_rows = [], []

    for row in rows:
      if question.match(row):
        true_rows.append(row)
      else:
        false_rows.append(row)
    
    

    return true_rows, false_rows


# Task 3: Best Split

> Estimated time: 30 minutes.

Now that we know how to split we need to determine where to split!
The best split is the one that maximizes the **information gain**, as determiend by the impurity measure.

$$
G_{d,\theta}(S) = Q(S) - \frac{|L_{d,\theta}|}{|S|}Q(L_{d,\theta}) - \frac{|R_{d,\theta}|}{|S|}Q(R_{d,\theta})
$$


Your task is to:
1. Complete the `find_best_split` function. 

 * You will need to combine several things: `compute_impurity`, `Question`, `partition`, and `info_gain`.
 * Hint: Structure your code as a double for-loop such that you examine whether each value for each feature is a good threshold.
    * Loop through all the features 
    * For each unique value of the current feature, create a `Question` and check if it is good.
    * Return the best information gain, and the question that lead to it. (or `0, None` if no good questions exist).

In [ ]:
def info_gain(left, right, current_uncertainty):
    """ Information Gain.
        The uncertainty of the starting node, minus the weighted impurity of two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * compute_impurity(left) - (1 - p) * compute_impurity(right)

In [ ]:
def find_best_split(rows):
    """ Find the best split by finding the information gain for each possible split.
    
        Returns: best_gain, best_question
    """
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep track of the feature / value that produced it
    current_uncertainty = compute_impurity(rows)
    n_features = len(rows[0]) - 1  # number of columns
    for i in range(n_features):
      uniq_vals = set()
      for row in rows:
        uniq_vals.add(row[i])
      for val in uniq_vals:
        q = Question(i, val)
        l, r = partition(rows, q)
        ig = info_gain(l, r, current_uncertainty)
        if ig > best_gain:
          best_gain = ig
          best_question = q
      

  
    ## YOUR CODE HERE
    
    
    return best_gain, best_question

print(find_best_split(data))


KeyError: ignored

# Task 4: Growing Trees

Now we have all the ingredients to start growing trees.
Have a look, and see if that works.


`build_tree` function is implemented using **recursion**.
Each iteration checks whether we shuold create a `Leaf_Node` (the base case) or it partitions the input date, and recursively calls itself on both partitions.
The result of the recursive function is saved as a `Decision_Node`.


`classify` is als implemented with recursion function.
The function takes **one** observation, and return the predicted class, by at each `Decision_Node` asking the `Question`.
Based on the answer follow the true-branch or the false-branch until you hit a `Leaf_Node`.

In [ ]:
class Leaf_Node:
    """ A Leaf_Node classifies data.
        This holds a dictionary of class (e.g., "Apple") -> number of times
        it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """ A Decision Node asks a question.
        This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def build_tree(rows):
    """ Builds the tree.
    """

    # Find the best split by finding the information gain for each possible split
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf_Node(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

my_tree = build_tree(data_train)

KeyError: ignored

In [ ]:
def classify(row, node):
    """ Recursively search
    """
    
    # Base Case:
    if isinstance(node, Leaf_Node):
        return node.predictions
    
    # Recursion: decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering (question.match)
    ## YOUR CODE HERE

    

# Visualizing Results

Now comes the fun part!
Run the cells below, and interpret the results.

Note that the Iris dataset is a quite simple dataset, and you shouldn't expect to get nearly this good results in general.

In [ ]:
def print_tree(node, spacing=""):
    """ World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf_Node):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

print_tree(my_tree)

In [ ]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

acc = 0
for row in data_test:
    pred = classify(row, my_tree)
    print ("Actual: %s. Predicted: %s" % (row[-1], print_leaf(pred)))
    
    if row[-1] in pred:
        acc += 1

print()
print('Accuracy:', acc/data_test.shape[0])

## Credits
This notebook is made heavily inspired and borrowing from: [Google Developers: Machine Learning Recipes #8](https://www.youtube.com/watch?v=LDRbO9a6XPU) ([source code](https://github.com/random-forests/tutorials/)).
